In [1]:
import requests
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity

url = 'https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/refs/heads/master/stopwords_ua.txt'
# за посиланням знаходиться список українських стоп-слів, який будемо парсити

response = requests.get(url)

if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'html.parser')
    stopwords_text = soup.get_text()
    stopwords = stopwords_text.split('\n')

In [2]:
from datasets import load_dataset

# завантаження датасету українських новин

ds = load_dataset("FIdo-AI/ua-news")

In [3]:
# використання перших 25000 статей із датасету

list_words = ds['train'][:25000]['text']

In [4]:
import re

def remove_stopwords(text, stop_words):
    
    # розбиваємо текст на слова
    
    words = text.split()
    
    # видаляємо апострофи, з якими виникають проблеми при обробці
    
    words = [word.lower().replace("'", "") for word in words] 
    
    # видаляємо стоп-слова та набори символів, які не містять літери
    
    filtered_words = [word for word in words if word not in stop_words and re.search(r'[a-zA-Zа-яА-Я]', word)]
    
    # повертаємо список слів після видалення стоп-слів
    
    return ' '.join(filtered_words)

# запуск видалення стоп-слів із текстів

filtered_texts = [remove_stopwords(text, stopwords) for text in list_words]

In [5]:
print(filtered_texts[999])

# приклад тексту після очищення

прес-секретар канцлера штеффен зайберт, повідомляє reuters. "канцлер жалем прийняла відома заяву відставку премєр-міністра італії. великою довірою працювала маттео ренці але, звичайно, демократичне рішення італійських виборців поважати, рішення премєр-міністра", зазначив зайберт. "німецький уряд пропонує тісну співпрацю партнерство наступному уряду італії, був", додав він. повідомлялося, премєр-міністр італії маттео ренці визнав поразку референдумі парламентської реформи оголосив відставку.


In [6]:
import re
from uk_stemmer import UkStemmer

stemmer = UkStemmer()
stemmed = []
# ініціалізуємо стеммер та створюємо список для речень після стеммінгу

def stem_text(text):
    # розбиваємо текст на список слів за допомогою регулярного виразу
    words = re.split(r'(\W)', text)
    
    # очищення списку від пустих слів
    
    words = [word for word in words if word != '']
    
    # застосування стеммінгу до всіх слів
    
    for i in range(len(words)):
        words[i] = stemmer.stem_word(words[i])
        
    # трансформуємо список слів назад до тексту
    stemmed_text = ''.join(words)
    return stemmed_text

# заповнюємо список стеммінгованих текстів

for i in filtered_texts:
    stemmed.append(stem_text(i))


In [7]:
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel

# Функція для обробки тексту
def preprocess(text):
    return [word for word in gensim.utils.simple_preprocess(text, min_len=3)]

# Підготовка даних до запуску
processed_docs = [preprocess(doc) for doc in stemmed]

# Створення словника на базі текстів
dictionary = corpora.Dictionary(processed_docs)

# Створення корпусу на базі текстів
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Перебір кількості тем
num_topics_range = range(2, 51)  # Від 2 до 20 тем
coherence_scores_cv = []
coherence_scores_umass = []

for num_topics in num_topics_range:
    lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=88, passes=10)

    # Обчислення c_v
    coherence_model_cv = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence_cv = coherence_model_cv.get_coherence()
    coherence_scores_cv.append(coherence_cv)

    # Обчислення u_mass
    coherence_model_umass = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='u_mass')
    coherence_umass = coherence_model_umass.get_coherence()
    coherence_scores_umass.append(coherence_umass)

    print(f'Кількість тем: {num_topics}, c_v: {coherence_cv}, u_mass: {coherence_umass}')


Кількість тем: 2, c_v: 0.5201855360913298, u_mass: -1.7057908424289119
Кількість тем: 3, c_v: 0.5476751166494545, u_mass: -1.7240644038510353
Кількість тем: 4, c_v: 0.5282610777161307, u_mass: -1.7677515194002273
Кількість тем: 5, c_v: 0.5352094569386523, u_mass: -1.7720728332953297
Кількість тем: 6, c_v: 0.545638771311185, u_mass: -2.136143966618373
Кількість тем: 7, c_v: 0.5499371160810859, u_mass: -1.8997710786350512
Кількість тем: 8, c_v: 0.5700266356224117, u_mass: -2.092466448915298
Кількість тем: 9, c_v: 0.5355440126963799, u_mass: -2.2734871811315305
Кількість тем: 10, c_v: 0.5435101901095755, u_mass: -2.082991134823779
Кількість тем: 11, c_v: 0.5441214953659089, u_mass: -2.166962672705336
Кількість тем: 12, c_v: 0.5926158739241498, u_mass: -2.114926629546121
Кількість тем: 13, c_v: 0.5690314406719366, u_mass: -2.5236234177936123
Кількість тем: 14, c_v: 0.5403963713188568, u_mass: -4.274516151104744
Кількість тем: 15, c_v: 0.5045730752023749, u_mass: -2.6382790125927147
Кількіс

In [8]:
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=22, random_state=88, passes=10)
top_topics = lda_model.top_topics(corpus)


In [9]:
top_topics

[([(0.006497719, 'країн'),
   (0.0064496063, 'нов'),
   (0.006187234, 'бізнес'),
   (0.0055981884, 'робот'),
   (0.0052660927, 'люд'),
   (0.0050762026, 'велик'),
   (0.00490006, 'так'),
   (0.0044705886, 'тог'),
   (0.0043755327, 'проблем'),
   (0.004012997, 'світ'),
   (0.0038732598, 'наприклад'),
   (0.0036659094, 'том'),
   (0.003485354, 'перш'),
   (0.0033119037, 'змін'),
   (0.0032577578, 'важлив'),
   (0.0030168486, 'питанн'),
   (0.002993706, 'час'),
   (0.0029793163, 'буд'),
   (0.0029690694, 'житт'),
   (0.0029623609, 'останн')],
  -1.56010730253303),
 ([(0.027594894, 'что'),
   (0.024445482, 'это'),
   (0.020357063, 'год'),
   (0.017120278, 'как'),
   (0.011594238, 'украин'),
   (0.010767241, 'которы'),
   (0.0081207035, 'работ'),
   (0.007903281, 'будет'),
   (0.0075039743, 'есл'),
   (0.0072636106, 'так'),
   (0.0066201286, 'компан'),
   (0.0065470585, 'тольк'),
   (0.0061529265, 'ест'),
   (0.0058920565, 'бизнес'),
   (0.005866326, 'сво'),
   (0.0056094415, 'дел'),
   (0.

In [10]:
# отримуємо розподіл тем для кожного документа в корпусі

doc_topic_distributions = [lda_model.get_document_topics(bow) for bow in corpus]

# функція для перетворення розподілу тем на вектор

def get_topic_vector (doc_topic_dist, num_topics):
    vec = [0] * num_topics
    
    # проходимо по кожній парі (індекс теми, ймовірність)
    
    for topic_idx, prob in doc_topic_dist:
        
        # встановлюємо ймовірність теми в відповідну позицію вектора
        
        vec[topic_idx] = prob
    return vec

In [11]:
# знаходимо для кожного документа вектор ймовірностей тем

topic_vectors = [get_topic_vector(doc, num_topics=40) for doc in doc_topic_distributions]

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# будуємо матрицю подібності, використовуючи cosine_similarity

similarity_matrix = cosine_similarity(topic_vectors)

In [13]:
import numpy as np

# функція для знаходження найбільш схожого тексту за індексом

def find_most_similar_for_given_text(similarity_matrix, text_index):
    similarities = similarity_matrix[text_index]
    
    # встановлюємо схожість тексту з самим собою на -1, щоб виключити з розгляду
    
    similarities[text_index] = -1
    # знаходимо індекс тексту з найбільшою семантичною схожістю
    most_similar_index = np.argmax(similarities)
    
    return most_similar_index

# обираємо текст, на який будемо шукати схожий

text_index = 21000
print(ds['train'][text_index]['text'])

most_similar_text_index = find_most_similar_for_given_text(similarity_matrix, text_index)

# знаходимо номер схожого тексту та виводимо сам текст

print(most_similar_text_index)
print(ds['train'][int(most_similar_text_index)]['text'])

 Випереджальне падіння імпорту дозволить Україні поліпшити зовнішньоторговельний баланс на $ 4 млрд.  Альфа-Банк Україна погіршив прогноз щодо динаміки українського ВВП, змінивши очікування зростання на 3,2% до падіння на 2%. Про це повідомляється в оновленому макропрогнозі банку.  За оцінками аналітиків, найбільш складним для української економіки стане другий квартал 2020 року. Падіння ВВП в цей період очікується на рівні 7%. У третьому кварталі спад продовжиться, а повноцінно відновитися від шоку економіка зможе лише в 2021 році. Прогноз банку передбачає досягнення Україною угоди про нову кредитну програму з Міжнародним валютним фондом у другому кварталі цього року. Транш МВФ разом із засобами інших міжнародних фінансових організацій допоможе Україні профінансувати основну частину різко збільшеного дефіциту державних фінансів, який досягне 8% ВВП в 2020 році, відзначають укладачі прогнозу. «Через різке розширення бюджетного дефіциту відношення державного боргу до ВВП України в 2020 